![alt text]( "OY Analytica")

# OY Analytica - An Open AI Community 
[github.com/oyanalytica](https://github.com/oyanalytica) 
<a href='https://twitter.com/oyanalytica'> </a>

In [7]:
import cv2
import numpy as np
import os

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces= face_classifier.detectMultiScale(gray,1.3,5)
    
    if faces is ():
        return None
    for (x,y,w,h) in faces:
        cropped_face =img[y:y+h, x:x+w]
    return cropped_face


cap = cv2.VideoCapture(0)
count = 0

while True:
    ret,photo = cap.read()
    if face_extractor(photo) is not None :
        count +=1
        face=cv2.resize(face_extractor(photo),(200,200))
        face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        
        file_name_path ='./faces/dataset2/'+str(count)+'.jpg'
        cv2.imwrite(file_name_path, face)
        
        cv2.putText(face, str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        cv2.imshow('face Cropped',face)
    
    else:
        count+=1
#         print("face "+str(count)+ " added")
        pass
    if cv2.waitKey(1)==13 or count == 200:
        break
    
cap.release()
cv2.destroyAllWindows()
print('Collecting Smaples Complete')

<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-7-94dcf8d8c7be>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In [8]:
cap.release()

In [9]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

data_path ="./faces/dataset2/"
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

Training_Data, Labels = [], []

for i, files in enumerate(onlyfiles):
    image_path = data_path +onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append((np.asarray(images, dtype=np.uint8)))
    Labels.append(i)
    

    
Labels = np.asarray(Labels, dtype = np.int32)

model = cv2.face_LBPHFaceRecognizer.create()
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model Trained Successfully")

Model Trained Successfully


In [10]:
import cv2
import numpy as np
import os
import pywhatkit
from datetime import datetime
import smtplib
import subprocess as sp
import time

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def whatsApp():
    now = datetime.now()
    hours = int(now.strftime("%H"))
    minutes = int(now.strftime("%M")) + 1
    pywhatkit.sendwhatmsg('+919568326846','This is face of Tinkal',hours,minutes)
    print("whatsApp message send")

def email():
    server = smtplib.SMTP_SSL("smtp.gmail.com",465)
    server.login("tinkal181921@dei.ac.in","Tinkal@9568")
    server.sendmail("Tinkal","tinkalshakya283125@gmail.com" , "this is face of Tinkal")
    server.quit()
    print("mail send")
    
def Instance_EBS():
    instance_id = sp.getoutput("aws ec2 run-instances --image-id ami-0ad704c126371a549 --instance-type t2.micro --count 1 --subnet-id subnet-0d4e2bb1599be8952 --security-group-ids sg-09a168c7c84e06582 --key-name MyfirstawsKey  --tag-specifications ResourceType=instance,Tags=[{Key=Name,Value=cvTask_Instance}]  --query Instances[*].[InstanceId] --output text")
    print("Instance Id : "+str(instance_id))
    volume_id = sp.getoutput("aws ec2 create-volume --volume-type gp2 --availability-zone ap-south-1a --size 5 --tag-specifications=ResourceType=volume,Tags=[{Key=Name,Value=cvTask_volume}] --query VolumeId --output text")
    print("Volume Id : "+str(volume_id) )
    time.sleep(20)
    sp.getoutput("aws ec2 attach-volume --volume-id {} --instance-id {} --device /dev/sdf".format(volume_id,instance_id))
    print("Volume and Instance Attached")
    
def face_detector(img, size=0.5):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
        
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        img1 = img[y:y+h, x:x+w]
        img1 = cv2.resize(img1, (200, 200))
    return img, img1

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()    
    image, face = face_detector(frame)
   
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        results = model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 90:
            cv2.putText(image, "Hey Tinkal", (250, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )      
            email()
            whatsApp()
            break     
         
        elif confidence <= 84 :
            cv2.putText(image, "Hey You are another person", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
#             Instance_EBS()
#             break
        
        else:
            cv2.putText(image, "I dont know, who r you", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: 
        break
        
cap.release()
cv2.destroyAllWindows()

<>:38: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:38: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-10-f90f5f3e4c99>:38: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


mail send
In 26 seconds web.whatsapp.com will open and after 20 seconds message will be delivered
whatsApp message send
